In [ ]:
import os
os.getcwd()

# Make sure server_pull.py file is in the same working directory

In [ ]:
# Make sure you are connected to NUS wifi or NUS VPN in order to import and run this function
# see the link below
# https://webvpn.nus.edu.sg/dana-na/auth/url_52/welcome.cgi

# Use server_pull_updated
#from server_pull_updated import pull
from server_pull import pull

# pull function has the following syntax: pull(user_group, secret_key, arm)
# please change 'test_user' and 'aaaaaaaa' to your own user_group and secret_key respectively

output = pull('user32','cnERvnBr',24)
#output = pull('user32','cnERvnBr',2)
output

# output comes as a dictionary with 4 key-value pairs; 
# Arm you pulled, Net Reward so far, How many times you have pulled, and reward from current pull

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

NetReward = 0
NetPull = 0
number_machine = 24

In [ ]:
def pull(user_group, secret_key, machine_num):
    global NetReward
    global NetPull
    
    NetReward = NetReward + 1
    NetPull = NetPull + 1
    
    return {'Arm': str(machine_num), 'NetReward': NetReward, 'Pull':NetPull, 'Reward':  np.random.randint(0, 100)}

In [ ]:
import random
import math

def adaptive_epsilon_greedy(user_group, password, given_rounds, number_machine):
    # Initialize the result dict with arms and statistics
    result = {idx + 1: {
                        "Arm": idx + 1,
                        "Output_Arm": 0,
                        "Total": 0, 
                        "Reward_List": [], 
                        "Total_Reward": 0,
                        } for idx in range(number_machine)
             }

    # Intial Setting
    initial_epsilon = 1.0  # Starting value for epsilon
    min_epsilon = 0.01  # Minimum value for epsilon
    epsilon_decay_rate = 0.005  # Decay rate for epsilon

    epsilon = initial_epsilon
    epsilons = []  # List of Chaning Epsilon
    
    # Learning Process for Exploitation / Exploration
    for num_round in range(1, given_rounds + 1):
        # Decide whether to explore or exploit
        if random.random() < epsilon:
            # Explore: randomly choose an arm
            arm = random.randint(1, number_machine)
            print(f"# {num_round}, Exploring Arm {arm}...")
        else:
            # Exploit: choose the arm with the highest average reward
            arm = max(result, key=lambda k: sum(result[k]["Reward_List"])/result[k]["Total"] if result[k]["Total"] > 0 else 0)
            print(f"# {num_round}, Exploiting Arm {arm}...")
        
        # Simulate pulling an arm and getting an output
        output = pull(user_group, password, arm)
        
        # Append Data
        result[arm]["Output_Arm"] = output["Arm"]
        result[arm]["Reward_List"].append(output["Reward"])
        result[arm]["Total"] += 1
        result[arm]["Total_Reward"] = sum(result[arm]["Reward_List"])
        
        # Decay epsilon using time-dependent exponential decay
        epsilon = max(min_epsilon, initial_epsilon * math.exp(-epsilon_decay_rate * num_round))
        epsilons.append(epsilon)
        
    return result


In [ ]:
# 'result'는 위에서 얻은 결과 딕셔너리입니다.

# Reset the each of arm as List (각 암의 라운드별 누적 보상을 저장할 리스트를 초기화)
cumulative_rewards = {arm: [0] for arm in result}

# Accumulate the reward (누적 보상)
for arm in result:
    cumulative = 0
    for reward in result[arm]["Reward_List"]:
        cumulative += reward
        cumulative_rewards[arm].append(cumulative)

# Ploting Graph (그래프)
plt.figure(figsize=(25, 15))
for arm in cumulative_rewards:
    plt.plot(cumulative_rewards[arm], label=f"Arm {arm}")

# Add title and label (그래프 제목 및 축 레이블을 추가)
plt.title("Cumulative Rewards of Each Arm Over Rounds")
plt.xlabel("Round")
plt.ylabel("Cumulative Reward")
plt.legend()
plt.show()
